In [1]:
HOPSWORKS_PROJECT_NAME = 'taxi_demand_1'

In [2]:
# autoreload
%load_ext autoreload
%autoreload 2

import os
from dotenv import load_dotenv

# set current working directory to root
os.chdir('..')

from src.paths import PARENT_DIR

# load key-value pairs from .env file
load_dotenv(PARENT_DIR /'.env')

HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']

In [3]:
from datetime import datetime
import pandas as pd
from src.data import load_raw_data

from_year = 2022
to_year = 2023
print(f'Loading raw data from {from_year} to {to_year}')

rides = pd.DataFrame()
for year in range(from_year, to_year+1):
    rides = pd.concat([rides, load_raw_data(year)])

Loading raw data from 2022 to 2023
File for 2022-01 already exists
File for 2022-02 already exists
File for 2022-03 already exists
File for 2022-04 already exists
File for 2022-05 already exists
File for 2022-06 already exists
File for 2022-07 already exists
File for 2022-08 already exists
File for 2022-09 already exists
File for 2022-10 already exists
File for 2022-11 already exists
File for 2022-12 not available 
File for 2023-01 not available 
File for 2023-02 not available 
File for 2023-03 not available 
File for 2023-04 not available 
File for 2023-05 not available 
File for 2023-06 not available 
File for 2023-07 not available 
File for 2023-08 not available 
File for 2023-09 not available 
File for 2023-10 not available 
File for 2023-11 not available 
File for 2023-12 not available 


In [4]:
print(f'{len(rides)=})')

len(rides)=36255131)


In [5]:
from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 262/262 [00:01<00:00, 208.95it/s]


In [6]:
# Backfill feature store
import hopsworks

project = hopsworks.login(project=HOPSWORKS_PROJECT_NAME, api_key_value=HOPSWORKS_API_KEY)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/20648


In [7]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [8]:
# save data to feature store
FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 1

feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

In [9]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Uploading Dataframe: 0.00% |          | Rows 0/2100192 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/20648/jobs/named/time_series_hourly_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x126ea0970>, None)